In [ ]:
import pandas as pd

# Load CSV
df = pd.read_csv(r'F:\Power BI visualization of time table\Time Table RA (Jan-June 2025) - B.Tech 4th sem.csv')

# Rename first column to 'Time'
df.rename(columns={df.columns[0]: 'Time'}, inplace=True)

# Melt to long format
df_melted = df.melt(id_vars=['Time'], var_name='Week Day', value_name='Class Details')

# Drop rows with Lunch or missing values
df_melted = df_melted[~df_melted['Class Details'].str.contains('Lunch', na=False)]
df_melted = df_melted.dropna(subset=['Class Details'])

# Split details safely
details_split = df_melted['Class Details'].str.split('|', expand=True)

# Fill missing columns
while details_split.shape[1] < 4:
    details_split[details_split.shape[1]] = 'Unknown'

# Assign fields
df_melted['Subject'] = details_split[0].str.strip()
df_melted['Teacher'] = details_split[1].str.strip() if 1 in details_split.columns else 'Unknown'
df_melted['Venue'] = details_split[2].str.strip() if 2 in details_split.columns else 'Unknown'
df_melted['Division'] = details_split[3].str.strip() if 3 in details_split.columns else 'All'

# Final cleaned dataframe
final_df = df_melted[['Week Day', 'Time', 'Subject', 'Teacher', 'Venue', 'Division']]

# Save to CSV
final_df.to_csv('formatted_timetable.csv', index=False)
print("Saved as 'formatted_timetable.csv'")


Saved as 'formatted_timetable.csv'


In [6]:
import pandas as pd

# Correct path to CSV file
file_path = r'F:\Power BI visualization of time table\Time Table RA (Jan-June 2025) - B.Tech 4th sem.csv'

# Read as CSV, not Excel
df = pd.read_csv(file_path, header=None, skiprows=1)

# Assign correct column names (adjust to your actual columns if needed)
df.columns = ["Day", "Time", "Subject", "Faculty", "Classroom", "Division", "NonTeaching", "Type", "Extra1", "Extra2"]

# Drop unnecessary rows: 1, 2, 3 and 14–32 (0-based: 0,1,2 and 13–31)
df = df.drop(index=list(range(0, 3)) + list(range(13, 32)), errors='ignore')

# Filter out Saturdays
df = df[~df["Day"].str.contains("Saturday", na=False)]

# Normalize values
df["Subject"] = df["Subject"].replace(["ELH", "FREE", "Free"], "Free")
df["Type"] = df["Type"].replace(["FREE", "Free"], "Free")
df["Faculty"] = df["Faculty"].fillna("NO")
df["NonTeaching"] = df["NonTeaching"].fillna("NO")
df["Type"] = df["Type"].fillna("Lec")
df["Division"] = df["Division"].fillna("ALL")

# Select only relevant columns
final_df = df[["Day", "Time", "Subject", "Faculty", "NonTeaching", "Type", "Division"]]

# Save to CSV
final_df.to_csv(r'F:\Power BI visualization of time table\cleaned_timetable.csv', index=False)

print("Cleaned timetable saved!")


Cleaned timetable saved!


In [ ]:
import pandas as pd
import re

# Load the original timetable CSV
file_path = 'your_input_file.csv'  # Change this to your actual file path
df = pd.read_csv(file_path)

# Set proper headers from the 3rd row (index 2)
df.columns = df.iloc[2]
df = df.iloc[3:].reset_index(drop=True)

# Rename the first column to "Day"
df.rename(columns={df.columns[0]: "Day"}, inplace=True)

# Melt into long format
df_melted = df.melt(id_vars=["Day"], var_name="Time", value_name="Entry")
df_melted.dropna(subset=["Entry"], inplace=True)

# Time slot order (used to duplicate lab entries)
time_slots = df.columns[1:].tolist()
next_slot_map = {time_slots[i]: time_slots[i + 1] for i in range(len(time_slots) - 1)}

# Function to parse each entry
def parse_entry(entry):
    entry = str(entry).strip()
    results = []

    if entry.startswith("RA"):
        segments = entry.split("\n")
        for seg in segments:
            match = re.match(r"(RA\d):\s*(.*)", seg)
            if match:
                division = match.group(1)
                content = match.group(2).strip()
                sub_match = re.match(r"(.*?)\s+\((.*?)\)", content)
                if sub_match:
                    subject, faculty = sub_match.groups()
                else:
                    subject, faculty = content, ""
                results.append((division, subject.strip(), faculty.strip()))
    else:
        match = re.match(r"(.*?)\s*\((.*?)\)", entry)
        if match:
            subject, faculty = match.groups()
        else:
            subject, faculty = entry, ""
        results.append(("ALL", subject.strip(), faculty.strip()))

    return results

# Expand entries and duplicate labs
expanded_rows = []
for _, row in df_melted.iterrows():
    parsed_data = parse_entry(row["Entry"])
    for division, subject, faculty in parsed_data:
        # Original time slot
        expanded_rows.append({
            "Day": row["Day"],
            "Time": row["Time"],
            "Division": division,
            "Subject": subject,
            "Faculty": faculty
        })
        # Duplicate if it's a lab and there's a next slot
        if division.startswith("RA") and row["Time"] in next_slot_map:
            next_time = next_slot_map[row["Time"]]
            expanded_rows.append({
                "Day": row["Day"],
                "Time": next_time,
                "Division": division,
                "Subject": subject,
                "Faculty": faculty
            })

# Create final DataFrame
df_final = pd.DataFrame(expanded_rows)

# Keep only valid weekdays
valid_days = ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'EVEN SAT', 'ODD SAT']
df_final = df_final[df_final['Day'].isin(valid_days)].reset_index(drop=True)

# Save the simplified timetable
output_path = 'simplified_timetable_expanded.csv'
df_final.to_csv(output_path, index=False)

print(f"Simplified timetable saved to: {output_path}")
